In [ ]:
from pedpy import load_trajectory
from pedpy import (
    Geometry,
    TrajectoryUnit,
    get_invalid_trajectory,
    is_trajectory_valid,
)
import pathlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import shapely
from shapely import LineString, Polygon
import warnings

warnings.filterwarnings("ignore")

# Setup geometry & measurement area

## Prepare geometry

![](../demos/uni-directional/geo.png)

In [ ]:
geometry_poly = Polygon([(-10, -3), (-10, 8), (10, 8), (10, -3)])

geometry = Geometry(walkable_area=geometry_poly)
geometry.add_obstacle(Polygon([(-9, -2), (-9, 0), (9, 0), (9, -2), (-9, -2)]))
geometry.add_obstacle(Polygon([(-9, 5), (-9, 7), (9, 7), (9, 5), (-9, 5)]))

In [ ]:
from pedpy.plotting.plotting import plot_geometry

plot_geometry(geometry=geometry, hole_color="lightgrey").set_aspect("equal")

**For demonstration purposes, wrongly place the obstacle s.th. some pedestrian walk through it!**

In [ ]:
geometry_faulty = Geometry(walkable_area=geometry_poly)
geometry_faulty.add_obstacle(
    Polygon([(-9, -2), (-9, 0.25), (9, 0.25), (9, -2), (-9, -2)])
)
geometry_faulty.add_obstacle(
    Polygon([(-9, 5), (-9, 7), (9, 7), (9, 5), (-9, 5)])
)

## Prepare measurement details

In [ ]:
ma = Polygon([(-1.5, 0), (-1.5, 5), (1.5, 5), (1.5, 0), (-1.5, 0)])
ml = LineString([(0, 0), (0, 5)])
passing_offset = 1.0

direction = np.array([-1, 0])

## Load trajectories

`pedpy` can load trajectories from text files, when:
- values are seperated by any whitespace, e.g., space, tab
- file has at least 5 columns in the following order: "ID", "frame", "X", "Y", "Z"
- file may contain comment lines with `#` at in the beginning

For meaningful analysis (and loading of the trajectory file) you also need
- unit of the trajectory (m or cm)
- frame rate

For recent experiments they are encoded in the header of the file, for older you may need to lead the documentation and provide the information in the loading process!

**Examples:**
With frame rate, but no unit
```
# description: UNI_CORR_500_01
# framerate: 25.00
#geometry: geometry.xml

# PersID	Frame	X	Y	Z
1	98	4.6012	1.8909	1.7600
1	99	4.5359	1.8976	1.7600
1	100	4.4470	1.9304	1.7600
...
```

No header at all:
```
1 27 164.834 780.844 168.937
1 28 164.835 771.893 168.937
1 29 163.736 762.665 168.937
1 30 161.967 753.088 168.937
...
```

In [ ]:
traj = load_trajectory(
    trajectory_file=pathlib.Path(
        "../demos/uni-directional/traj_UNI_CORR_500_01.txt"
    ),
    default_unit=TrajectoryUnit.METER,  # needs to be provided as it not defined in the file
    # default_frame_rate=25., # can be ignored here as the frame rate is defined in the file
)

## Plot setup

In [ ]:
from pedpy import plot_measurement_setup

fig = plt.figure(figsize=(15, 20))
ax1 = fig.add_subplot(111, aspect="equal")
# ax1 = plot_geometry(geometry=geometry_faulty, ax=ax1) # remove comment to show the borders of the faulty geometry

plot_measurement_setup(
    traj=traj,
    geometry=geometry,
    measurement_areas=[ma],
    measurement_lines=[ml, shapely.offset_curve(ml, passing_offset)],
    traj_width=0.1,
    traj_start_marker=".",
    hole_color="lightgrey",
    ml_color="b",
    ma_color="g",
    ma_alpha=0.1,
    ma_line_color="g",
    ax=ax1,
)
plt.show()

## Validate that trajectory is completely inside the walkable area.

In [ ]:
print(
    f"Trajectory is valid: {is_trajectory_valid(traj=traj, geometry=geometry)}"
)
get_invalid_trajectory(traj=traj, geometry=geometry)

In [ ]:
print(
    f"Trajectory is valid: {is_trajectory_valid(traj=traj, geometry=geometry_faulty)}"
)
get_invalid_trajectory(traj=traj, geometry=geometry_faulty)

# Filter the trajectory data

## Filter by geometrical predicates

### Data inside Polygon

In [ ]:
data_inside_ma = traj.data[shapely.within(traj.data.points, ma)]
data_inside_ma

### Data outside Polygon

In [ ]:
data_outside_ma = traj.data[~shapely.within(traj.data.points, ma)]
data_outside_ma

### Data close to Polygon

In [ ]:
data_close_ma = traj.data[shapely.dwithin(traj.data.points, ma, 1)]
data_close_ma

## Get all data points in a frame range

In [ ]:
data_frame_range = traj.data[
    traj.data.frame.between(300, 400, inclusive="both")
]
data_frame_range

# Density

## Classic density

In [ ]:
from pedpy import compute_classic_density

classic_density = compute_classic_density(
    traj_data=traj.data, measurement_area=ma
)
classic_density

In [ ]:
classic_density.reset_index().plot.line(x="frame", y="classic density")

## Voronoi density

### Compute individual Voronoi Polygons

#### Without cut-off

In [ ]:
from pedpy import compute_individual_voronoi_polygons

individual = compute_individual_voronoi_polygons(
    traj_data=traj.data, geometry=geometry
)

#### With cut-off
**Note:** second argument of `cut_off` needs to be divisable by 4!

In [ ]:
from pedpy import compute_individual_voronoi_polygons

individual_cutoff = compute_individual_voronoi_polygons(
    traj_data=traj.data, geometry=geometry, cut_off=(1.0, 12)
)

### Compute actual Voronoi density

#### Without cut-off

In [ ]:
from pedpy import compute_voronoi_density

density_voronoi, intersecting = compute_voronoi_density(
    individual_voronoi_data=individual, measurement_area=ma
)

In [ ]:
density_voronoi.reset_index().plot.line(x="frame", y="voronoi density")

#### With cut-off

In [ ]:
from pedpy import compute_voronoi_density

density_voronoi_cutoff, intersecting_cutoff = compute_voronoi_density(
    individual_voronoi_data=individual_cutoff, measurement_area=ma
)

In [ ]:
density_voronoi_cutoff.reset_index().plot.line(x="frame", y="voronoi density")

## Comparision

In [ ]:
fig = plt.figure(figsize=(10, 6))
plt.plot(
    classic_density.reset_index().frame,
    classic_density["classic density"].values,
    label="classic",
    lw=3,
)
plt.plot(
    density_voronoi.reset_index().frame,
    density_voronoi["voronoi density"],
    label="voronoi",
    lw=3,
)
plt.plot(
    density_voronoi_cutoff.reset_index().frame,
    density_voronoi_cutoff["voronoi density"],
    label="voronoi cutoff",
    lw=3,
)
plt.xlabel("frame")
plt.ylabel("rho / 1/m^2")
plt.legend()
plt.grid()
plt.show()

## Plot voronoi cells

In [ ]:
from pedpy import plot_voronoi_cells

frame_start = 1200

for frame in range(frame_start, frame_start + 100, 20):
    fig = plt.figure(f"frame = {frame}", figsize=(15, 20))
    fig.suptitle(f"frame = {frame}", y=0.62, fontsize=20)
    df_frame = intersecting[intersecting.frame == frame]
    df_frame = pd.merge(traj.data, df_frame, on=["ID", "frame"])

    ax1 = fig.add_subplot(121, aspect="equal")
    ax1.set_title("w/o cutoff")
    plot_voronoi_cells(
        data=df_frame,
        geometry=geometry,
        color_mode="id",
        show_ped_positions=True,
        ped_size=10,
        ax=ax1,
    )

    df_frame_cutoff = intersecting_cutoff[intersecting_cutoff.frame == frame]
    df_frame_cutoff = pd.merge(traj.data, df_frame_cutoff, on=["ID", "frame"])

    ax2 = fig.add_subplot(122, aspect="equal")
    ax2.set_title("w cutoff")

    plot_voronoi_cells(
        data=df_frame_cutoff,
        geometry=geometry,
        color_mode="id",
        show_ped_positions=True,
        ped_size=10,
        ax=ax2,
    )

    fig.tight_layout()
    plt.show()

## Passing density (individual)

In [ ]:
from pedpy import compute_passing_density
from pedpy import compute_frame_range_in_area

frames_in_area, _ = compute_frame_range_in_area(
    traj_data=traj.data, measurement_line=ml, width=passing_offset
)
passing_density = compute_passing_density(
    density_per_frame=classic_density, frames=frames_in_area
)
passing_density

# Velocity

## Individual speed

In [ ]:
from pedpy import compute_individual_velocity

individual_speed = compute_individual_velocity(
    traj_data=traj.data,
    frame_rate=traj.frame_rate,
    frame_step=5,
    x_y_components=True,
)
individual_speed

In [ ]:
individual_speed.plot.scatter(x="frame", y="speed")

In [ ]:
individual_speed_direction = compute_individual_velocity(
    traj_data=traj.data,
    frame_rate=traj.frame_rate,
    frame_step=5,
    movement_direction=direction,
    x_y_components=True,
)
individual_speed_direction

In [ ]:
individual_speed_direction.plot.scatter(x="frame", y="speed")

## Mean speed (in measurement area)

In [ ]:
from pedpy import compute_mean_velocity_per_frame

mean_speed = compute_mean_velocity_per_frame(
    traj_data=traj.data,
    measurement_area=ma,
    individual_velocity=individual_speed,
)
mean_speed

In [ ]:
individual_speed

In [ ]:
mean_speed.reset_index().plot.line(x="frame", y="speed")

In [ ]:
mean_speed_direction = compute_mean_velocity_per_frame(
    traj_data=traj.data,
    measurement_area=ma,
    individual_velocity=individual_speed_direction,
)
mean_speed_direction

## Voronoi speed

In [ ]:
from pedpy import compute_voronoi_velocity

In [ ]:
individual_voronoi = intersecting.copy(deep=True)

In [ ]:
voronoi_velocity = compute_voronoi_velocity(
    traj_data=traj.data,
    individual_voronoi_intersection=individual_voronoi,
    individual_velocity=individual_speed,
    measurement_area=ma,
)
voronoi_velocity

In [ ]:
voronoi_velocity.reset_index().plot.line(x="frame", y="voronoi speed")

In [ ]:
voronoi_velocity_direction = compute_voronoi_velocity(
    traj_data=traj.data,
    individual_voronoi_intersection=individual_voronoi,
    individual_velocity=individual_speed_direction,
    measurement_area=ma,
)
voronoi_velocity_direction

## Comparison mean velocity vs voronoi velocity

In [ ]:
fig = plt.figure(figsize=(8, 6))
plt.plot(
    voronoi_velocity.reset_index().frame, voronoi_velocity, label="voronoi"
)
plt.plot(
    voronoi_velocity_direction.reset_index().frame,
    voronoi_velocity_direction,
    label="voronoi direction",
)
plt.plot(mean_speed.reset_index().frame, mean_speed, label="classic")
plt.plot(
    mean_speed_direction.reset_index().frame,
    mean_speed_direction,
    label="classic direction",
)
plt.xlabel("frame")
plt.ylabel("v / m/s")
plt.legend()
plt.grid()
plt.show()

## Passing speed (individual)

In [ ]:
from pedpy import compute_passing_speed
from pedpy import compute_frame_range_in_area

frames_in_area, _ = compute_frame_range_in_area(
    traj_data=traj.data, measurement_line=ml, width=passing_offset
)
passing_speed = compute_passing_speed(
    frames_in_area=frames_in_area,
    frame_rate=traj.frame_rate,
    distance=passing_offset,
)
passing_speed

# Flow

## N-t diagram

In [ ]:
from pedpy import compute_n_t

nt, crossing = compute_n_t(
    traj_data=traj.data, measurement_line=ml, frame_rate=traj.frame_rate
)

In [ ]:
nt.plot(x="Time [s]")

## Flow

In [ ]:
from pedpy import compute_flow

delta_t = 100
flow = compute_flow(
    nt=nt,
    crossing_frames=crossing,
    individual_speed=individual_speed,
    delta_t=delta_t,
    frame_rate=traj.frame_rate,
)
flow

# Neighborhood

In [ ]:
from pedpy import compute_neighbors

neighbors = compute_neighbors(individual_cutoff)
neighbors

In [ ]:
frame = 231

voronoi_neighbors = pd.merge(
    individual_cutoff[individual_cutoff.frame == frame],
    neighbors[neighbors.frame == frame],
    on=["ID", "frame"],
)

for base in voronoi_neighbors["ID"].values[8:10]:
    base_neighbors = voronoi_neighbors[voronoi_neighbors["ID"] == base][
        "neighbors"
    ].values[0]

    fig = plt.figure(f"frame = {frame}", figsize=(20, 10))

    ax = fig.add_subplot(111, aspect="equal")
    fig.suptitle(f"id = {base}")
    plot_geometry(ax=ax, geometry=geometry)

    for _, row in voronoi_neighbors.iterrows():
        poly = row["individual voronoi"]
        ped_id = row["ID"]

        are_neighbors = ped_id in base_neighbors

        color = "gray"
        if ped_id == base:
            color = "green"

        if are_neighbors:
            color = "red"

        ax.plot(*poly.exterior.xy, alpha=1, color=color)
        ax.fill(*poly.exterior.xy, alpha=0.5, color=color)

    fig.tight_layout()
    plt.show()

# Profiles

In [ ]:
from pedpy import (
    compute_profiles,
    compute_individual_voronoi_polygons,
    compute_individual_velocity,
    VelocityMethod,
)

In [ ]:
min_frame_profiles = 900
max_frame_profiles = 1000

frames_data = traj.data[
    traj.data.frame.isin(range(min_frame_profiles, max_frame_profiles))
]

individual_frames = compute_individual_voronoi_polygons(
    traj_data=frames_data,
    geometry=geometry,
    cut_off=(0.8, 12),
)

individual_speed = compute_individual_velocity(
    traj_data=frames_data,
    frame_rate=traj.frame_rate,
    frame_step=5,
)

In [ ]:
density_profiles, velocity_profiles = compute_profiles(
    individual_voronoi_velocity_data=pd.merge(
        individual_frames, individual_speed, on=["ID", "frame"]
    ),
    walkable_area=geometry.walkable_area,
    grid_size=0.2,
    velocity_method=VelocityMethod.VORONOI,
)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

bounds = geometry.walkable_area.bounds

fig, (ax0, ax1) = plt.subplots(nrows=1, ncols=2, figsize=(10, 10))

ax0.set_title("Density")
cm = ax0.imshow(
    np.mean(density_profiles, axis=0),
    extent=[bounds[0], bounds[2], bounds[1], bounds[3]],
    interpolation="None",
    cmap="jet",
    vmin=0,
    vmax=0.5,
)
fig.colorbar(cm, ax=ax0, shrink=0.3)
ax0.plot(*geometry.walkable_area.exterior.xy, color="w")

ax1.set_title("Velocity")
cm = ax1.imshow(
    np.mean(velocity_profiles, axis=0),
    extent=[bounds[0], bounds[2], bounds[1], bounds[3]],
    cmap="jet_r",
    vmin=0,
    vmax=1.5,
)
fig.colorbar(cm, ax=ax1, shrink=0.3)

ax1.plot(*geometry.walkable_area.exterior.xy, color="w")

fig.tight_layout()

# What to do with the results?

## Combine multiple DataFrames


In [ ]:
traj.data

In [ ]:
individual

In [ ]:
data_with_voronoi_cells = traj.data.merge(intersecting, on=["ID", "frame"])
data_with_voronoi_cells

In [ ]:
data_with_voronoi_cells_speed = data_with_voronoi_cells.merge(
    individual_speed[["ID", "frame", "speed"]], on=["ID", "frame"]
)
data_with_voronoi_cells_speed

## Save in files

### Create directories to store the results

In [ ]:
pathlib.Path("results_introduction/profiles/velocity").mkdir(
    parents=True, exist_ok=True
)
pathlib.Path("results_introduction/profiles/density").mkdir(
    parents=True, exist_ok=True
)

results_directory = pathlib.Path("results_introduction")

### Save Pandas dataframe (result from everything but profiles) as csv

In [ ]:
import csv

data_with_voronoi_cells_speed["individual density"] = shapely.area(
    data_with_voronoi_cells_speed["individual voronoi"]
)

with open(
    results_directory / "individual_result.csv", "w"
) as individual_output_file:
    individual_output_file.write(f"#framerate:	{traj.frame_rate}\n\n")
    data_with_voronoi_cells_speed[
        [
            "ID",
            "frame",
            "X",
            "Y",
            "Z",
            "individual density",
            "speed",
            "individual voronoi",
            "intersection voronoi",
        ]
    ].to_csv(
        individual_output_file,
        mode="a",
        header=True,
        sep="\t",
        index_label=False,
        index=False,
        quoting=csv.QUOTE_NONNUMERIC,
    )

### Save numpy arrays (result from profiles) as txt

In [ ]:
results_directory_density = results_directory / "profiles/density"
results_directory_velocity = results_directory / "profiles/velocity"

for i in range(len(range(min_frame_profiles, min_frame_profiles + 10))):
    frame = min_frame_profiles + i
    np.savetxt(
        results_directory_density / f"density_frame_{frame:05d}.txt",
        density_profiles[i],
    )
    np.savetxt(
        results_directory_velocity / f"velocity_frame_{frame:05d}.txt",
        velocity_profiles[i],
    )